Лабораторна Робота Лабораторна робота №1. Підготовка статистичних даних: читання, описова статистика, очистка (КН-2-3)

 Підготовка статистичних даних: генерація, описова статистика, очистка

ВАРІАНТ 009 — Нерухомість (листинги та закриті угоди) підготував студент групи кн 22 Лісовський Данило Олександрович.



In [2]:
install.packages("tidyverse")
install.packages("janitor")
install.packages("naniar")
install.packages("stringdist") 
install.packages("scales") 


trying URL 'https://cran.rstudio.com/bin/windows/contrib/4.5/janitor_2.2.1.zip'
Content type 'application/zip' length 292867 bytes (286 KB)
downloaded 286 KB



package ‘janitor’ successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Користувач\AppData\Local\Temp\RtmpYFGTe4\downloaded_packages


trying URL 'https://cran.rstudio.com/bin/windows/contrib/4.5/naniar_1.1.0.zip'
Content type 'application/zip' length 2776098 bytes (2.6 MB)
downloaded 2.6 MB



package ‘naniar’ successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Користувач\AppData\Local\Temp\RtmpYFGTe4\downloaded_packages


trying URL 'https://cran.rstudio.com/bin/windows/contrib/4.5/stringdist_0.9.15.zip'
Content type 'application/zip' length 717761 bytes (700 KB)
downloaded 700 KB



package ‘stringdist’ successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Користувач\AppData\Local\Temp\RtmpYFGTe4\downloaded_packages


trying URL 'https://cran.rstudio.com/bin/windows/contrib/4.5/scales_1.4.0.zip'
Content type 'application/zip' length 878250 bytes (857 KB)
downloaded 857 KB



package ‘scales’ successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Користувач\AppData\Local\Temp\RtmpYFGTe4\downloaded_packages


Установив необхідні бібліотеки,  tidyverse  -> dplyr, ggplot2, tidyr, readr, janitor -> очистка даних, naniar -> # аналіз пропусків, stringdist -> порівняння текстів, scales -> форматування графіків.

In [3]:
library(tidyverse)

SEED <- 42
set.seed(SEED)

RECORD_COUNT <- 3000

# Довідники
CITIES <- list(
  Kyiv = list(
    districts = c('Shevchenkivskyi', 'Pecherskyi', 'Holosiivskyi', 'Obolonskyi',
                  'Podilskyi', 'Darnytskyi', 'Dniprovskyi', 'Desnianskyi',
                  'Solomianskyi', 'Sviatoshynskyi'),
    coords = list(lat = 50.4501, lon = 30.5234, spread = 0.15)
  ),
  Lviv = list(
    districts = c('Halytskyi', 'Zaliznychnyi', 'Lychakivskyi', 'Sykhivskyi',
                  'Frankivskyi', 'Shevchenkivskyi'),
    coords = list(lat = 49.8397, lon = 24.0297, spread = 0.08)
  ),
  Odesa = list(
    districts = c('Prymorskyi', 'Malynovskyi', 'Kiiivskyi', 'Suvorovskyi'),
    coords = list(lat = 46.4825, lon = 30.7233, spread = 0.10)
  )
)

STREET_PREFIXES <- c('Shevchenka', 'Franka', 'Lesi Ukrainky', 'Khreshchatyk',
                     'Bandery', 'Sichovykh Striltsiv', 'Heroiv', 'Peremohy',
                     'Mira', 'Soborna', 'Teatralna', 'Parkova')

AGENCIES <- c('EstateGroup', 'RealtyPro', 'HomeCenter', 'CityRealty',
              'PrimeEstate', 'UrbanHouse', NA)

SOURCES <- c('portal_a', 'portal_b', 'portal_c')

EXCHANGE_RATES <- list(UAH = 1, USD = 37.5, EUR = 40.8)

# Функції
generate_coordinates <- function(city_name) {
  city <- CITIES[[city_name]]$coords
  lat <- city$lat + (runif(1) - 0.5) * city$spread
  lon <- city$lon + (runif(1) - 0.5) * city$spread
  return(list(lat = round(lat, 6), lon = round(lon, 6)))
}

generate_area_total <- function() {
  return(round(rlnorm(1, meanlog = 4.2, sdlog = 0.4)))
}

generate_date_range <- function(start_date, end_date) {
  days_between <- as.numeric(difftime(end_date, start_date, units = "days"))
  random_days <- sample(0:days_between, 1)
  return(start_date + random_days)
}

cat(sprintf("Генерація %d записів з seed=%d...\n", RECORD_COUNT, SEED))

records <- list()
listing_id_counter <- 100000

for (i in 1:RECORD_COUNT) {
  city_name <- sample(names(CITIES), 1)
  district <- sample(CITIES[[city_name]]$districts, 1)
  
  street <- sample(STREET_PREFIXES, 1)
  building_num <- sample(1:150, 1)
  address <- sprintf("vul. %s, %d", street, building_num)
  
  coords <- generate_coordinates(city_name)
  lat <- coords$lat
  lon <- coords$lon
  
  rooms <- sample(1:5, 1)
  area_total <- generate_area_total()
  area_kitchen <- round(area_total * runif(1, 0.12, 0.25))
  
  floors_total <- sample(5:25, 1)
  floor <- sample(1:floors_total, 1)
  
  deal_type <- ifelse(runif(1) < 0.6, 'rent', 'sale')
  
  currency <- sample(c('UAH', 'USD', 'EUR'), 1)
  
  if (deal_type == 'rent') {
    if (currency == 'UAH') {
      price <- round(area_total * runif(1, 180, 320))
    } else if (currency == 'USD') {
      price <- round(area_total * runif(1, 5, 9))
    } else {
      price <- round(area_total * runif(1, 4.5, 8))
    }
  } else {
    if (currency == 'UAH') {
      price <- round(area_total * runif(1, 35000, 55000))
    } else if (currency == 'USD') {
      price <- round(area_total * runif(1, 950, 1450))
    } else {
      price <- round(area_total * runif(1, 900, 1350))
    }
  }
  
  price_uah <- round(price * EXCHANGE_RATES[[currency]])
  
  start_date <- as.Date("2024-01-01")
  end_date <- as.Date("2024-06-30")
  published_at <- generate_date_range(start_date, end_date)
  
  rand_status <- runif(1)
  if (rand_status < 0.35) {
    status <- 'closed'
    closed_at <- published_at + sample(3:60, 1)
    closed_at_str <- format(closed_at, "%Y-%m-%d %H:%M:%S")
  } else if (rand_status < 0.85) {
    status <- 'active'
    closed_at_str <- ""
  } else {
    status <- 'withdrawn'
    if (runif(1) < 0.5) {
      closed_at <- published_at + sample(1:45, 1)
      closed_at_str <- format(closed_at, "%Y-%m-%d %H:%M:%S")
    } else {
      closed_at_str <- ""
    }
  }
  
  agency <- sample(AGENCIES, 1)
  agent_id <- ifelse(!is.na(agency), sprintf("AG%d", sample(1000:9999, 1)), "")
  phone <- sprintf("+380%d%d", sample(50:99, 1), sample(1000000:9999999, 1))
  
  source <- sample(SOURCES, 1)
  
  descriptions <- c(
    sprintf("Comfortable %d-room apartment in %s district. Total area %d sq.m.", 
            rooms, district, area_total),
    sprintf("Great location in %s. %d rooms, %d floor. Well renovated.", 
            city_name, rooms, floor),
    sprintf("Spacious apartment near metro. %d sq.m, %d rooms. Excellent condition.", 
            area_total, rooms),
    sprintf("Modern design, all amenities. %s, %d sq.m.", 
            district, area_total),
    sprintf("Perfect for families. %d-bedroom apartment, %d/%d floor.", 
            rooms, floor, floors_total)
  )
  description <- sample(descriptions, 1)
  
  record <- tibble(
    listing_id = sprintf("L%d", listing_id_counter),
    published_at = format(published_at, "%Y-%m-%d %H:%M:%S"),
    closed_at = closed_at_str,
    status = status,
    deal_type = deal_type,
    city = city_name,
    district = district,
    address = address,
    lat = lat,
    lon = lon,
    rooms = rooms,
    floor = floor,
    floors_total = floors_total,
    area_total = area_total,
    area_kitchen = area_kitchen,
    price = price,
    currency = currency,
    price_uah = price_uah,
    agent_id = agent_id,
    agency = ifelse(is.na(agency), "", as.character(agency)),
    contact_phone = phone,
    source = source,
    description = description
  )
  
  records[[i]] <- record
  listing_id_counter <- listing_id_counter + 1
  
  if (i %% 1000 == 0) {
    cat(sprintf("  Згенеровано %d записів...\n", i))
  }
}

df <- bind_rows(records)

# Створення
dir.create("../clean", recursive = TRUE, showWarnings = FALSE)

output_file <- sprintf('../clean/clean_realestate_seed%d_n%d.csv', SEED, RECORD_COUNT)
write_csv(df, output_file)

Генерація 3000 записів з seed=42...
  Згенеровано 1000 записів...
  Згенеровано 2000 записів...
  Згенеровано 3000 записів...


Отже згенеровано чисті данні в розмірі 3000 в яких містяться необхідні поля та чистий датасет за умовою


In [8]:
library(tidyverse)
library(lubridate)

# Налаштування
SEED <- 42
set.seed(SEED)

# Завантаження чистих даних
df <- read_csv('../clean/clean_realestate_seed42_n3000.csv', show_col_types = FALSE)
df_dirty <- df

# Визначення записів для забруднення
DIRTY_RANGE <- 0.65
n_dirty_records <- as.integer(nrow(df_dirty) * DIRTY_RANGE)
dirty_indices <- sample(1:nrow(df_dirty), n_dirty_records, replace = FALSE)

fields_to_null <- list(
  price_uah = 0.10,
  lat = 0.12,
  lon = 0.12,
  area_kitchen = 0.15,
  agent_id = 0.08,
  closed_at = 0.05
)

for (field in names(fields_to_null)) {
  if (field %in% colnames(df_dirty)) {
    pct <- fields_to_null[[field]]
    n_nulls <- as.integer(n_dirty_records * pct)
    null_indices <- sample(dirty_indices, min(n_nulls, length(dirty_indices)), replace = FALSE)
    df_dirty[[field]][null_indices] <- NA
  }
}

# Додавання повних дублів
n_dupes <- as.integer(n_dirty_records * 0.025)
dupe_indices <- sample(dirty_indices, min(n_dupes, length(dirty_indices)), replace = FALSE)
dupes <- df_dirty[dupe_indices, ]
df_dirty <- bind_rows(df_dirty, dupes)

# Змішування форматів дат
date_cols <- c('published_at', 'closed_at')
for (date_col in date_cols) {
  if (date_col %in% colnames(df_dirty)) {
    for (idx in dirty_indices) {
      if (idx <= nrow(df_dirty) && !is.na(df_dirty[[date_col]][idx]) && runif(1) < 0.5) {
        tryCatch({
          dt <- as.Date(df_dirty[[date_col]][idx])
          df_dirty[[date_col]][idx] <- format(dt, "%d.%m.%Y")
        }, error = function(e) {})
      }
    }
  }
}

# Зміна регістру категорій
cat_cols <- c('status', 'deal_type', 'city')
for (col in cat_cols) {
  if (col %in% colnames(df_dirty)) {
    for (idx in dirty_indices) {
      if (idx <= nrow(df_dirty)) {
        val <- df_dirty[[col]][idx]
        if (!is.na(val)) {
          rand <- runif(1)
          if (rand < 0.3) {
            df_dirty[[col]][idx] <- toupper(val)
          } else if (rand < 0.6) {
            df_dirty[[col]][idx] <- tolower(val)
          } else if (rand < 0.8) {
            df_dirty[[col]][idx] <- str_to_title(val)
          }
        }
      }
    }
  }
}

# Опечатки в названнях міст
city_typos <- list(
  'Kyiv' = c('Kiyv', 'Kyiv ', ' Kyiv', 'Kiev'),
  'Lviv' = c('Lviv ', 'Lwiv', ' Lviv'),
  'Odesa' = c('Odessa', 'Odesa ', ' Odesa')
)

for (idx in dirty_indices) {
  if (idx <= nrow(df_dirty) && runif(1) < 0.1) {
    city <- df_dirty$city[idx]
    if (!is.na(city)) {
      for (correct in names(city_typos)) {
        if (grepl(tolower(correct), tolower(city))) {
          df_dirty$city[idx] <- sample(city_typos[[correct]], 1)
          break
        }
      }
    }
  }
}

# Подвійні пробіли в адресах
if ('address' %in% colnames(df_dirty)) {
  for (idx in dirty_indices) {
    if (idx <= nrow(df_dirty) && runif(1) < 0.15) {
      addr <- df_dirty$address[idx]
      if (!is.na(addr)) {
        df_dirty$address[idx] <- str_replace_all(addr, ' ', '  ')
      }
    }
  }
}

# Переплутування координат lat/lon
if ('lat' %in% colnames(df_dirty) && 'lon' %in% colnames(df_dirty)) {
  n_swap <- as.integer(n_dirty_records * 0.05)
  swap_indices <- sample(dirty_indices, min(n_swap, length(dirty_indices)), replace = FALSE)
  for (idx in swap_indices) {
    if (idx <= nrow(df_dirty) && !is.na(df_dirty$lat[idx]) && !is.na(df_dirty$lon[idx])) {
      lat <- df_dirty$lat[idx]
      lon <- df_dirty$lon[idx]
      df_dirty$lat[idx] <- lon
      df_dirty$lon[idx] <- lat
    }
  }
}

# Логічні помилки: floor > floors_total
if ('floor' %in% colnames(df_dirty) && 'floors_total' %in% colnames(df_dirty)) {
  n_floor_errors <- as.integer(n_dirty_records * 0.03)
  error_indices <- sample(dirty_indices, min(n_floor_errors, length(dirty_indices)), replace = FALSE)
  for (idx in error_indices) {
    if (idx <= nrow(df_dirty) && !is.na(df_dirty$floors_total[idx])) {
      df_dirty$floor[idx] <- df_dirty$floors_total[idx] + sample(1:5, 1)
    }
  }
}

# Аномальні площі: дуже малі або дуже великі
if ('area_total' %in% colnames(df_dirty)) {
  n_area_errors <- as.integer(n_dirty_records * 0.02)
  error_indices <- sample(dirty_indices, min(n_area_errors, length(dirty_indices)), replace = FALSE)
  for (idx in error_indices) {
    if (idx <= nrow(df_dirty)) {
      if (runif(1) < 0.5) {
        df_dirty$area_total[idx] <- sample(5:11, 1)
      } else {
        df_dirty$area_total[idx] <- sample(501:999, 1)
      }
    }
  }
}

# Некоректні ціни: нулі та від'ємні значення
price_col <- NULL
if ('price' %in% colnames(df_dirty)) {
  price_col <- 'price'
} else if ('price_uah' %in% colnames(df_dirty)) {
  price_col <- 'price_uah'
}

if (!is.null(price_col)) {
  n_price_errors <- as.integer(n_dirty_records * 0.015)
  error_indices <- sample(dirty_indices, min(n_price_errors, length(dirty_indices)), replace = FALSE)
  for (idx in error_indices) {
    if (idx <= nrow(df_dirty)) {
      if (runif(1) < 0.5) {
        df_dirty[[price_col]][idx] <- 0
      } else {
        df_dirty[[price_col]][idx] <- -sample(100:10000, 1)
      }
    }
  }
}

# Ціни як текст з пробілами
if (!is.null(price_col)) {
  for (idx in dirty_indices) {
    if (idx <= nrow(df_dirty) && runif(1) < 0.2 && !is.na(df_dirty[[price_col]][idx])) {
      price <- as.integer(df_dirty[[price_col]][idx])
      df_dirty[[price_col]][idx] <- format(price, big.mark = " ", trim = TRUE)
    }
  }
}

# Збереження забруднених даних
dir.create('../raw', showWarnings = FALSE, recursive = TRUE)
write_csv(df_dirty, '../raw/generated_dirty.csv')